In [1]:
from utils_v1 import *


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates,
    PA, PADeforestationTime, PADistanceMap, PAMultipleDates,
    MTDeforestationTime,
    MTMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential

from src.trainer.evidential import TrainerEvidential

from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.trainer.evidential as evidential


In [3]:

config = {
	"training": False,
	"save_probabilities": False,
	"loadInference": False,
	"addPastDeforestationInput": True,
	"inference_times": 1,
	"Uncertainty": {
		"annealing_step": 10*375/2 # 1875
	}
}




'''
if config["training"] == True:
	dates = [2017, 2018]
else:
	dates = [2018, 2019]
if config["training"] == True:
	dates = [2018, 2019]
else:
	dates = [2019, 2020]
'''
# dates = [2018, 2019]
# dates = [2017, 2018]

# dataset = PAMultipleDates(dates = dates, # dates = [2017, 2018, 2019]. [2018, 2019]. [2015, 2016, 2017, 2018]
# 	addPastDeforestationInput = config['addPastDeforestationInput'],
# 	borderBuffer = 2)	
dates = [2018, 2019]
# dates = [2019, 2020]

dataset = MTMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
	addPastDeforestationInput = config['addPastDeforestationInput'],
	borderBuffer = 2)	
# dataset = MTDeforestationTime()	
# [2016, 2017, 2018]

ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerEvidential(
		dataset)
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [4]:
[10*375//1, 10*375//1.5, 10*375//2, 10*375//3, 10*375//4]


[3750, 2500.0, 1875, 1250, 937]

In [5]:
# if config["training"] == True:
# experiment_values = [10*375//1, 10*375//1.5, 10*375//2, 10*375//3, 10*375//4]
# experiment_values = [10*375//3, 10*375//3, 10*375//3, 10*375//3, 10*375//3]
# experiment_values = [10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2, 10*375//2]
# experiment_values = [10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375]
experiment_values = [10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375, 10*375]

experiment_values = [int(x) for x in experiment_values]
results = []

#exp_id_base = 148
# exp_id_base = 155
exp_id_base = 175


exp_ids = range(exp_id_base, exp_id_base+len(experiment_values))


In [6]:
exp_ids

range(191, 201)

In [ ]:
from enum import Enum
UncertaintyMethod = Enum('Predictor', 'pred_var MI pred_entropy KL pred_entropy_single evidential')
uncertaintyMethod = "evidential"


In [7]:


for experiment_value, exp_id in zip(experiment_values, exp_ids):
    config = {
        "training": True,
    	"learning_rate": 1e-4,
        "weights": [0.1, 0.9, 0],
        "save_probabilities": False,
        "loadInference": False,
        "addPastDeforestationInput": True,
        "inference_times": 1,
        "Uncertainty": {
            "annealing_step": experiment_value# 10*375/2 # 1875
        },
    	"uncertainty_method": uncertaintyMethod,
        "evidence_function": evidential.relu_evidence
    }

    trainer = TrainerEvidential(config, dataset, patchesHandler)

    trainer.defineExperiment(exp_id)

    trainer.setExperimentPath()
    trainer.createLogFolders()

    trainer.loadDataset()

    # %%
    if config["training"] == True:
        trainer.run()

    # %%
    result = trainer.run_predictor()
    results.append(result)


    with open('log.pkl', 'wb') as f:
        pickle.dump(results, f)

D:/Jorge/datasets/deforestation/MT/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([161124576,   1332784,    240796,    338545,    313901,    287191,
                                                                       708581,    648966,    876898,   1007787,   1098901,   1054092,
                                                                      2096062,   2473828,   1400992], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

ic| self.predicted_unpad.shape: (16795, 10420)
    self.predicted.shape: (16795, 10420)
ic| predicted_test.shape: (70001560,)
ic| predicted_test.shape: (39774655,)
ic| np.unique(predicted, return_counts=True): (array([0.], dtype=float32), array([175003900], dtype=int64))
    np.unique(predicted_larger_than_min_area, return_counts=True): (array([0.], dtype=float32), array([175003900], dtype=int64))
ic| np.unique(ignored_polygons, return_counts=True): (array([0.], dtype=float32), array([175003900], dtype=int64))
c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
ic| self.f1: 0.0, self.precision: 0.0, self.recall: 0.0
ic| self.label_mask_val.shape: (21000468,)
ic| self.label_mask_val_valid.shape: (10906966,)
c:\Users\jchamorro\Anaconda3\

(array([0., 1.], dtype=float32), array([173103734,   1900166], dtype=int64))


ic| self.label_mask.shape: (16795, 10420)
ic| self.mask_amazon_ts.shape: (16795, 10420)
ic| self.label_mask_test.shape: (70001560,)
ic| self.error_mask_test.shape: (39774655,)
ic| bounds: (0.0015, 0.9985)
c:\Users\jchamorro\Anaconda3\envs\tf2\lib\site-packages\scipy\optimize\_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


0.38232011321635484
threshold 0.38232011321635484
(0,) (0,)
cm_correct []


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
print(results)

[{'metrics': {'precision_L': array([nan]), 'recall_L': array([0.]), 'recall_Ltotal': array([0.]), 'AA': array([0.03000042]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'f1_L': array([nan]), 'f1_H': array([nan]), 'f1': 0.0}, 'metrics_audited': {'precision': array([1.]), 'recall': array([0.87966265]), 'f1': array([0.93597928])}, 'exp': 181}, {'metrics': {'precision_L': array([0.9266887]), 'recall_L': array([0.84085823]), 'recall_Ltotal': array([0.50764546]), 'AA': array([0.02999879]), 'precision_H': array([0.75952801]), 'recall_H': array([0.31876432]), 'f1_L': array([0.88168954]), 'f1_H': array([0.44906269]), 'f1': 73.97}, 'metrics_audited': {'precision': array([0.95746098]), 'recall': array([0.90392245]), 'f1': array([0.92992176])}, 'exp': 182}, {'metrics': {'precision_L': array([0.94748293]), 'recall_L': array([0.89449666]), 'recall_Ltotal': array([0.63664633]), 'AA': array([0.03000292]), 'precision_H': array([0.64688705]), 'recall_H': array([0.43562819]), 'f1_L': array([0.9

In [ ]:
trainer.f1

78.95

In [ ]:
print(trainer.f1)

78.95
